In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob,os
import sys
import scipy
from importlib import  reload
from time import process_time 
#from libraries.lib_gather_data import get_hhid_FIES

%load_ext autoreload

In [2]:
from shock_libraries import *
from plotting_libraries import *
from response_libraries import get_response_sp
#
from income_shock_libraries_ps import *
#
from libraries.lib_country_dir import set_directories, load_survey_data, get_places_dict
from libraries.lib_get_hh_savings import get_hh_savings
from libraries.pandas_helper import broadcast_simple

In [3]:
# formatting & aesthetics
font = {'family':'sans serif', 'size':10}
plt.rc('font', **font)
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.facecolor'] = 'white'
sns.set_style("white")

sns_pal = sns.color_palette('Set1', n_colors=8, desat=.4)
greys_pal = sns.color_palette('Greys', n_colors=9)

In [4]:
## create crosstabs for sectors for K:

# load file:
df = pd.read_csv('./csv/lfs_a09_pqkb_ranked_20200420.csv')
df

#pd.crosstab(s1, b1, dropna=False)
df_xtab = pd.crosstab(df['a09_pqkb'], df['sector'], dropna=False)
df_xtab.to_csv('./temp/sector_crosstab_20200420.csv')

In [5]:
# ## create crosstabs for sectors for K:


# mr = merge_rank()

#     # get subset: a09_pqkb
# df = mr[['hhid_lfs','cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home']]

# # load file:
# #df = pd.read_csv('./csv/lfs_a09_pqkb_ranked_20200420.csv')
# #df

# #pd.crosstab(s1, b1, dropna=False)
# df_xtab = pd.crosstab(df['c19_pclass'], df['LFS_sector'], dropna=False)
# df_xtab.to_csv('./temp/sector_c19_crosstab_20200420.csv')

In [6]:
#rand_weighted_shock_3dim_v2()

In [7]:
x = merge_rank()
x

,hhid_lfs,cc101_lno,affected_cat,region,province,hhexp,cash_abroad,regft,hhwgt,fsize,...,newempstat,j12c09_grade,pwgt,prov_code,LFS_sector,income_loss,pwgt_shock,sector_y,w_home,demand_scale
0,101011011112,4,a,41,10,749221,0,496,627.90253,9.0,...,1.0,634.0,626.6919,10,professional_services,0,0.0,professional_services,4.0,0.0
1,101011011112,6,a,41,10,749221,0,496,627.90253,9.0,...,1.0,Third Year College,638.0756,10,government,0,0.0,government,1.0,1.0
2,101011011112,8,a,41,10,749221,0,496,627.90253,9.0,...,1.0,Third Year College,719.1034,10,manufacturing,0,0.0,manufacturing,0.0,0.0
3,101011013134,1,a,41,10,279725,15000,56445,627.90253,4.0,...,1.0,High School Graduate,450.7739,10,government,0,0.0,government,1.0,1.0
4,101011013134,2,a,41,10,279725,15000,56445,627.90253,4.0,...,1.0,High School Graduate,626.6919,10,food_entertainment,0,0.0,food_entertainment,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75987,984350483,6,a,12,98,116547,6000,645,392.69931,9.0,...,1.0,No Grade Completed,470.8087,98,ag,0,0.0,ag,2.0,1.0
75988,984350483,9,a,12,98,116547,6000,645,392.69931,9.0,...,1.0,Elementary Graduate,351.3405,98,ag,0,0.0,ag,2.0,0.0
75989,9843505110,2,a,12,98,111267,10000,835,392.69931,6.0,...,1.0,No Grade Completed,238.6860,98,retail,0,0.0,retail,0.0,1.0
75990,9843505110,3,a,12,98,111267,10000,835,392.69931,6.0,...,1.0,Second Year High School,483.5432,98,ag,0,0.0,ag,2.0,1.0


In [8]:
x.pcwgt

0        612.839956
1        612.839956
2        612.839956
3        817.119967
4        817.119967
            ...    
75987    373.098711
75988    373.098711
75989    559.648075
75990    559.648075
75991    559.648075
Name: pcwgt, Length: 75992, dtype: float64

In [9]:
sum(x.pcwgt)

39511432.36139427

In [10]:
#---------------------------------- Added: 20200422: <PS>

#def rand_weighted_shock_3dim_v2():

"""    
    Updated 20200422 
        - incorporate 3rd dimension for sector (public/private/gov) and impact on essentiality
            - now, only enforce that government jobs are maintained, across all sectors
    
    Updated 20200419 
        - incorporate 3rd dimension for sector (public/private/gov) and impact on essentiality 
    Updated 20200413:
        - incorporate 2nd dimension for social distancing potential
    module added to covid_phl: <income_shock_libraries_ps.py>

    primary development: <FCT> rand_weigthed_shock_distance() <FCT>

    - function to replace: 
        rand_weighted_shock_3dim()--> rand_weighted_shock_distance() --> rand_weighted_shock_1() --> get_income_shock(): in <shock_libraries.py>

    - description:
        * matches existing df_shock dataframe (compatibiility)
        * uses Kayenat table of job descriptions demand value for 'a09_pqkb' by sector to create weighted probability of job disruption by sector, as input to 'fa' column of df_shock             
        dataframe -- representative FIES and LFS data 

       * for values 0.0, 0.5,1 : assigns each job description a random: 0-50%, 50-99%, 100% chance of disruption, weighting each by the prevalence of that role in each sector, to generate cumulative probability of disruption. 


        * now incorporates enforcement of social distancing measures, by enforcing social distance in non-essential jobs based on K.Kabirs' 0-4 'work-from-home' scoring.



"""
# develop 3 factor code here:

# make each factor modular

mr = merge_rank()

    # get subset: a09_pqkb
mr_subset = mr[['hhid_lfs','cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home']]

indexNames = mr_subset[mr_subset['a09_pqkb'] == 'nan' ].index

    # Delete these row indexes from dataFrame
mr_subset.drop(indexNames , inplace=True)
mr_subset = mr_subset.reset_index(drop=True)

 # get subset: c19_pclass

indexNames2 = mr_subset[mr_subset['c19_pclass'] == 'nan' ].index

    # Delete these row indexes from dataFrame
mr_subset.drop(indexNames2 , inplace=True)
mr_subset = mr_subset.reset_index(drop=True)

# make new column of combined string a09 && c19:
mr_subset['a09c19'] = mr_subset['a09_pqkb'] +'-'+mr_subset['c19_pclass']

    # enforce string:
mr_subset['a09_pqkb'] = [str(q) for q in mr_subset['a09_pqkb']] # enforce type = string
mr_subset['LFS_sector'] = [str(q) for q in mr_subset['LFS_sector']] # enforce type = string
mr_subset['c19_pclass'] = [str(q) for q in mr_subset['c19_pclass']] # enforce type = string
mr_subset['a09c19'] = [str(q) for q in mr_subset['a09c19']] # enforce type = string


    # generate fraction by sector
mr_subset['desc_count'] = mr_subset.groupby('a09_pqkb')['a09_pqkb'].transform('count')# count unique jobs and append to mr_subset
mr_subset['sector_count'] = mr_subset.groupby('LFS_sector')['LFS_sector'].transform('count') #count total unique sectors and append to mr_subset
mr_subset['sector_frac'] = mr_subset['desc_count'] / mr_subset['sector_count'] # get fraction of sector as weighting

# scale fractions by weight to synthesize individuals: pcwgt
mr_subset['desc_count_pcwgt'] = mr_subset['pcwgt']*mr_subset['desc_count']


#####
# here, need to insert a new column that merges a09 and c19 -- done
# then, drop duplicates off of this column, so that we can minimize computation

# still need logic to build the logic for each job sector
## may need to restructure this whole section of code

#####
    # drop duplicates (now that overall weighting established)
mr_subset = mr_subset.drop_duplicates(subset='a09_pqkb')
mr_subset = mr_subset.reset_index(drop=True)



    # generate probability and combine with relative weighting
mr_subset['partial_prob'] = np.nan
mr_subset['third_col'] = np.nan
mr_subset['dummy'] = np.nan

    # incorporate Kayenat tables into 'di' &&
    # nested logic to incorporate 0-4 scale for social distancing measures
    ## where scores of 0 & 1 result in complete job lost, due to unable to distance
i=0
while i < len(mr_subset):



    if mr_subset.demand_scale[i] == 0:

            # incorporate 0-4 scale logic:

        if mr_subset.w_home[i] == 0:
            mr_subset.partial_prob[i] = 0

        elif mr_subset.w_home[i] == 1:
            mr_subset.partial_prob[i] = 0

        else:
            mr_subset.partial_prob[i] = mr_subset.sector_frac[i] * (random.randint(0,50)/100)


    elif mr_subset.demand_scale[i] == 0.5: 

            # incorporate 0-4 scale logic:
        if mr_subset.w_home[i] == 0:
            mr_subset.partial_prob[i] = 0

        elif mr_subset.w_home[i] == 1:
            mr_subset.partial_prob[i] = 0

        else: 
            mr_subset.partial_prob[i] = mr_subset.sector_frac[i] * (random.randint(50,100)/100)

    elif mr_subset.demand_scale[i] == 1.0:
        mr_subset.partial_prob[i] = mr_subset.sector_frac[i]
    else:
        mr_subset.dummy[i] = -99


    # incorporate 3rd column modifiers here:
    if (mr_subset['c19_pclass'][i] == "Gov't/Gov't Corporation"):
        mr_subset.partial_prob[i] = mr_subset.sector_frac[i]  # essentially reverts the random uniform logic implemented above


    i = i + 1

    # remove nans in summing fields, and dummy storage
del mr_subset['dummy']

    #define shock table:
shock_null = { 'ag':           [  0,  0],
                 'mining':        [  0,  0],
                 'utilities':     [  0,  0],
                 'construction':  [0.0,1.0],
                 'manufacturing': [0.0,1.0],
                 'wholesale':     [0.0,1.0],
                 'retail':        [0.0,1.0],
                 'transportation':[0.0,1.0],
                 'information':   [0.0,1.0],
                 'finance':       [0.0,1.0],
                 'professional_services':[0.0,1.0],
                 'eduhealth':     [0.0,1.0],
                 'food_entertainment':[0.0,1.0],
                 'government':    [  0,  0],
                 'other':         [0.0,1.0]}
df_shock_null = pd.DataFrame(data=shock_null).T
df_shock_null.columns = ['fa','di']
df_shock_null.index.name = 'LFS_sector'


df_shock_cum = df_shock_null

    # get mean probability by sector:

for seclist in df_shock_cum.index: # hard-coded to existing shock table

    pp = mr_subset[mr_subset.LFS_sector == seclist]
    p4 = 1 - sum(pp.partial_prob)

        # build shock table:
    df_shock_cum['fa'][seclist] = df_shock_cum['fa'][seclist] + p4

    # save to separate var for testing    
rand_weighted_shock = df_shock_cum

#return(rand_weighted_shock)
rand_weighted_shock

/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/l

,fa,di
LFS_sector,,
ag,1.644808e-02,0.0
mining,8.608736e-01,0.0
utilities,1.965517e-01,0.0
construction,7.183911e-01,1.0
manufacturing,7.435769e-01,1.0
wholesale,6.796806e-01,1.0
retail,3.319488e-01,1.0
transportation,9.356626e-01,1.0
information,1.110223e-16,1.0


In [11]:
mr_subset['desc_count_pcwgt'] = mr_subset['pcwgt'] * mr_subset['desc_count']



KeyError: 'pcwgt'

In [12]:
x = merge_rank()

In [13]:
x.pcwgt

0        612.839956
1        612.839956
2        612.839956
3        817.119967
4        817.119967
            ...    
75987    373.098711
75988    373.098711
75989    559.648075
75990    559.648075
75991    559.648075
Name: pcwgt, Length: 75992, dtype: float64

In [19]:

    # generate fraction by sector
x['desc_count'] = x.groupby('a09_pqkb')['a09_pqkb'].transform('count')# count unique jobs and append to mr_subset
x['sector_count'] = x.groupby('LFS_sector')['LFS_sector'].transform('count') #count total unique sectors and append to mr_subset
x['sector_frac'] = x['desc_count'] / x['sector_count'] # get fraction of sector as weighting

# scale fractions by weight to synthesize individuals: pcwgt
x['desc_count_pcwgt'] = x['pcwgt']*x['desc_count']


x['sector_count_pcwgt'] = x['pcwgt'] * x['sector_count']
x['sector_frac_pcgwt'] = x['desc_count_pcwgt'] / x['sector_count_pcwgt']

In [24]:
x.desc_count_pcwgt
x.sector_count_pcwgt
sum(x.sector_frac_pcgwt, skipna=True)

TypeError: sum() takes no keyword arguments

In [25]:
x.sector_frac_pcgwt.sum(skipna=True)

21191.48893808978